En este notebook se construye un conjunto de test (básicamente este conjunto será la última semana de enero) y se proporcionan algunas utilidades:

- Una función que dada una predicción la pase al formato apropiado para la entrega
- Una función que dada una predicción devuelva la métrica sobre el conjunto empleado como test (que en este caso será la última semana de Enero)

In [5]:
import pandas as pd

df = pd.read_pickle('../data/data_v1.pkl') 
df.head()

,ID,SAMPLETIME,DELTA,READING,DATE,TIME
0,0,2019-06-13 08:34:09,17.0,369320.0,2019-06-13,08:34:09
1,0,2019-06-13 17:34:10,2.0,369403.0,2019-06-13,17:34:10
2,0,2019-06-13 18:34:10,0.0,369403.0,2019-06-13,18:34:10
3,0,2019-06-13 04:34:10,1.0,369284.0,2019-06-13,04:34:10
4,0,2019-06-13 14:34:10,28.0,369356.0,2019-06-13,14:34:10


In [6]:
df = df.drop(['SAMPLETIME','TIME','READING'],axis=1)
df = df.groupby(['ID','DATE']).sum().reset_index()
df.head()

,ID,DATE,DELTA
0,0,2019-02-01,243.0
1,0,2019-02-02,236.0
2,0,2019-02-03,335.0
3,0,2019-02-04,252.0
4,0,2019-02-05,220.0


In [13]:
df[df['ID']==0].tail(7)

,ID,DATE,DELTA
358,0,2020-01-25,390.0
359,0,2020-01-26,304.0
360,0,2020-01-27,213.0
361,0,2020-01-28,232.0
362,0,2020-01-29,403.0
363,0,2020-01-30,425.0
364,0,2020-01-31,255.0


In [28]:
penultimos_7_dias = df[df['ID']==0].tail(7)
penultimos_7_dias[ultimos_7_dias['DATE']=="2020-01-25"].iloc[0]['DELTA']

390.0

### Última semana de Enero (deberíamos usarla como semana 2 de la predicción)

In [32]:
test_dict = {'ID':[],"2020-01-25":[],"2020-01-26":[],"2020-01-27":[],
             "2020-01-28":[],"2020-01-29":[],"2020-01-30":[],"2020-01-31":[]}
january_last_week = ["2020-01-25","2020-01-26","2020-01-27","2020-01-28","2020-01-29","2020-01-30","2020-01-31"]

for i in df['ID'].unique():
    test_dict['ID'].append(i)
    ultimos_7_dias = df[df['ID']==i].tail(7)
    for day in january_last_week:
        if day in ultimos_7_dias['DATE'].values:
            test_dict[day].append(ultimos_7_dias[ultimos_7_dias['DATE']==day].iloc[0]['DELTA'])
        else:
            test_dict[day].append(0)

In [ ]:
test_dict = {'ID':[],"2020-01-25":[],"2020-01-26":[],"2020-01-27":[],
             "2020-01-28":[],"2020-01-29":[],"2020-01-30":[],"2020-01-31":[]}
january_last_week = ["2020-01-25","2020-01-26","2020-01-27","2020-01-28","2020-01-29","2020-01-30","2020-01-31"]

for i in df['ID'].unique():
    test_dict['ID'].append(i)
    ultimos_7_dias = df[df['ID']==i].tail(7)
    for day in january_last_week:
        if day in ultimos_7_dias['DATE'].values:
            test_dict[day].append(ultimos_7_dias[ultimos_7_dias['DATE']==day].iloc[0]['DELTA'])
        else:
            test_dict[day].append(0)

In [46]:
test = pd.DataFrame.from_dict(test_dict)
test.columns = ['ID','Dia_1','Dia_2','Dia_3','Dia_4','Dia_5','Dia_6','Dia_7']
test['Semana_1'] = test[['Dia_1','Dia_2','Dia_3','Dia_4','Dia_5','Dia_6','Dia_7']].sum(axis=1)
test

,ID,Dia_1,Dia_2,Dia_3,Dia_4,Dia_5,Dia_6,Dia_7,Semana_1
0,0,390.00,304.00,213.0,232.0,403.00,425.0,255.00,2222.00
1,1,1.32,2.97,4.0,1.0,1.98,10.0,32.95,54.22
2,2,30.00,32.00,39.0,40.0,45.00,27.0,59.00,272.00
3,3,639.00,350.00,289.0,380.0,404.00,339.0,391.00,2792.00
4,4,272.04,281.98,263.6,303.4,421.00,420.0,253.98,2216.00
...,...,...,...,...,...,...,...,...,...
2742,2746,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00
2743,2747,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00
2744,2748,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00
2745,2749,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00


In [ ]:
'''
esta función espera dos dataframes con el mismo formato que el df de arriba, 
devuelve el mse tal y como se calcula en el enunciado del datathon
'''
def compute_mse(pred, real):
    